# 取り組み
+ Stx2大型欠失3アレルで以下のエラーが出たのでデバックをする
    + preprocess/correct_sequence_errorの`replace_atmark`におけるエラーのよう

```
Traceback (most recent call last):
  File "/mnt/d/Research/DAJIN2/misc/quickstart.py", line 145, in <module>
    preprocess.correct_sequence_error.execute(TEMPDIR, FASTA_ALLELES, CONTROL_NAME, SAMPLE_NAME)
  File "/mnt/d/Research/DAJIN2/src/DAJIN2/core/preprocess/correct_sequence_error.py", line 135, in execute
    cssplits_sample_atmark_replaced = replace_atmark(cssplits_sample_error_replaced, sequence)
  File "/mnt/d/Research/DAJIN2/src/DAJIN2/core/preprocess/correct_sequence_error.py", line 103, in replace_atmark
    cssplits_replaced[idx][i] = random.choice(cssplits_sampling_all)
  File "/home/kuno/miniconda/lib/python3.10/random.py", line 378, in choice
    return seq[self._randbelow(len(seq))]
IndexError: list index out of range
```

In [1]:
# ルートディレクトリをPathに含めるおまじない
import sys, os
from pathlib import Path
if os.getcwd() != "/mnt/d/Research/DAJIN2":
    parent_path = str(Path(os.path.dirname(os.path.abspath("__file__"))).parent.parent)
    sys.path.append(parent_path)
    os.chdir(parent_path)
print(os.getcwd())

/mnt/d/Research/DAJIN2


# 実験

In [5]:
from src.DAJIN2.core import preprocess, classification, clustering, consensus, report

# # * 2-cut deletion
SAMPLE, CONTROL, ALLELE, NAME, GENOME, DEBUG, THREADS = (
    "tests/data/knockout/test_barcode25.fq.gz",
    "tests/data/knockout/test_barcode30.fq.gz",
    "tests/data/knockout/design_stx2.fa",
    "test-knockout",
    "mm10",
    True,
    14,
)

In [6]:
##########################################################
# Check inputs
##########################################################
preprocess.check_inputs.check_files(SAMPLE, CONTROL, ALLELE)
TEMPDIR = Path("DAJINResults", ".tempdir", NAME)
IS_CACHE_CONTROL = preprocess.check_inputs.exists_cached_control(CONTROL, TEMPDIR)
IS_CACHE_GENOME = preprocess.check_inputs.exists_cached_genome(GENOME, TEMPDIR, IS_CACHE_CONTROL)
UCSC_URL, GOLDENPATH_URL = None, None
if GENOME and not IS_CACHE_GENOME:
    UCSC_URL, GOLDENPATH_URL = preprocess.check_inputs.check_and_fetch_genome(GENOME)

##########################################################
# Format inputs
##########################################################
SAMPLE_NAME = preprocess.format_inputs.extract_basename(SAMPLE)
CONTROL_NAME = preprocess.format_inputs.extract_basename(CONTROL)
FASTA_ALLELES = preprocess.format_inputs.dictionize_allele(ALLELE)

preprocess.format_inputs.make_directories(TEMPDIR, SAMPLE_NAME, CONTROL_NAME)

if GENOME:
    GENOME_COODINATES = preprocess.format_inputs.fetch_coodinate(GENOME, UCSC_URL, FASTA_ALLELES["control"])
    CHROME_SIZE = preprocess.format_inputs.fetch_chrom_size(GENOME_COODINATES["chr"], GENOME, GOLDENPATH_URL)
    preprocess.format_inputs.cache_coodinates_and_chromsize(TEMPDIR, GENOME, GENOME_COODINATES, CHROME_SIZE)


In [7]:
for identifier, sequence in FASTA_ALLELES.items():
        contents = "\n".join([">" + identifier, sequence]) + "\n"
        output_fasta = Path(TEMPDIR, "fasta", f"{identifier}.fasta")
        output_fasta.write_text(contents)

In [8]:
for path_fasta in Path(TEMPDIR, "fasta").glob("*.fasta"):
        name_fasta = path_fasta.stem
        preprocess.mappy_align.output_sam(TEMPDIR, path_fasta, name_fasta, CONTROL, CONTROL_NAME, threads=THREADS)
        preprocess.mappy_align.output_sam(TEMPDIR, path_fasta, name_fasta, SAMPLE, SAMPLE_NAME, threads=THREADS)
        preprocess.mappy_align.output_sam(
            TEMPDIR, path_fasta, name_fasta, CONTROL, CONTROL_NAME, preset="splice", threads=THREADS
        )
        preprocess.mappy_align.output_sam(
            TEMPDIR, path_fasta, name_fasta, SAMPLE, SAMPLE_NAME, preset="splice", threads=THREADS
        )

In [9]:
for path_sam in Path(TEMPDIR, "sam").glob(f"{CONTROL_NAME}_splice_*"):
        preprocess.calc_midsv.output_midsv(TEMPDIR, path_sam)
for path_sam in Path(TEMPDIR, "sam").glob(f"{SAMPLE_NAME}_splice_*"):
        preprocess.calc_midsv.output_midsv(TEMPDIR, path_sam)

In [10]:
###############################################################################
# Correct CSSPLITS
###############################################################################
preprocess.correct_revititive_deletions.execute(TEMPDIR, FASTA_ALLELES, CONTROL_NAME, SAMPLE_NAME)
preprocess.correct_sequence_error.execute(TEMPDIR, FASTA_ALLELES, CONTROL_NAME, SAMPLE_NAME)
preprocess.correct_knockin.execute(TEMPDIR, FASTA_ALLELES, CONTROL_NAME, SAMPLE_NAME)

IndexError: list index out of range

In [14]:
from __future__ import annotations

import random
import re
from collections import Counter, defaultdict
from copy import deepcopy
from pathlib import Path
import midsv
from scipy import stats
from scipy.spatial.distance import cosine


def set_indexes(sequence: str):
    sequence_length = len(sequence)
    num_subset = sequence_length % 5
    left_idx = 0
    right_idx = sequence_length
    if num_subset == 1:
        left_idx += 1
    elif num_subset == 2:
        left_idx += 1
        right_idx -= 1
    elif num_subset == 3:
        left_idx += 2
        right_idx -= 1
    elif num_subset == 4:
        left_idx += 2
        right_idx -= 2
    return left_idx, right_idx


def count_indels_5mer(cssplits: list[list[str]], left_idx: int, right_idx: int) -> list[dict]:
    transposed = [list(t) for t in zip(*cssplits)]
    count_indels_5mer = []
    for i in range(left_idx, right_idx, 5):
        count = {"ins": [1] * 5, "del": [1] * 5, "sub": [1] * 5}
        cssplits_5mer = transposed[i : i + 5]
        for j, cs in enumerate(cssplits_5mer):
            counter = Counter(cs)
            for key, cnt in counter.items():
                if key.startswith("=") or key == "N" or re.search(r"a|c|g|t|n", key):
                    continue
                if key.startswith("+"):
                    count["ins"][j] += cnt
                elif key.startswith("-"):
                    count["del"][j] += cnt
                elif key.startswith("*"):
                    count["sub"][j] += cnt
        count_indels_5mer.append(count)
    return count_indels_5mer


def extract_sequence_errors(count_5mer_sample, count_5mer_control, coverage_sample, coverage_control):
    sequence_errors = [set() for _ in range(len(count_5mer_sample))]
    for i in range(len(sequence_errors)):
        for mutation in ["ins", "del", "sub"]:
            samp = [c / coverage_sample for c in count_5mer_sample[i][mutation]]
            cont = [c / coverage_control for c in count_5mer_control[i][mutation]]
            distance = 1 - cosine(samp, cont)
            _, pvalue = stats.ttest_ind(samp, cont, equal_var=False)
            if distance > 0.95 and pvalue > 0.05:
                sequence_errors[i].add(mutation)
    return sequence_errors


def replace_errors_to_atmark(cssplits_sample, sequence_errors, left_idx, right_idx):
    cssplits_replaced = []
    for samp in cssplits_sample:
        samp_replaced = deepcopy(samp)
        for idx_error, idx_5mer in enumerate(range(left_idx, right_idx, 5)):
            samp_5mer = samp[idx_5mer : idx_5mer + 5]
            error = sequence_errors[idx_error]
            if "ins" in error:
                samp_5mer = ["@" if cs.startswith("+") else cs for cs in samp_5mer]
            if "del" in error:
                samp_5mer = ["@" if cs.startswith("-") else cs for cs in samp_5mer]
            if "sub" in error:
                samp_5mer = ["@" if cs.startswith("*") else cs for cs in samp_5mer]
            samp_replaced[idx_5mer : idx_5mer + 5] = samp_5mer
        cssplits_replaced.append(samp_replaced)
    return cssplits_replaced


def replace_atmark(cssplits: list[list[str]], sequence: str) -> list[list[str]]:
    random.seed(1)
    cssplits_replaced = deepcopy(cssplits)
    sequence_length = len(sequence)
    for i in range(1, sequence_length - 1):
        cssplits_atmark = defaultdict(str)
        cssplits_sampling_key = defaultdict(list)
        cssplits_sampling_all = []
        for idx, cssplit in enumerate(cssplits):
            key = ",".join([cssplit[i - 1], cssplit[i + 1]])
            if cssplit[i] == "@":
                cssplits_atmark[idx] = key
            else:
                cssplits_sampling_key[key].append(cssplit[i])
                cssplits_sampling_all.append(cssplit[i])
        for idx, key in cssplits_atmark.items():
            if cssplits_sampling_key[key]:
                cssplits_replaced[idx][i] = random.choice(cssplits_sampling_key[key])
            else:
                cssplits_replaced[idx][i] = random.choice(cssplits_sampling_all)
    for cs in cssplits_replaced:
        if cs[0] == "@":
            cs[0] = "=" + sequence[0]
        if cs[-1] == "@":
            cs[-1] = "=" + sequence[-1]
    return cssplits_replaced


###############################################################################
# main
###############################################################################


def execute(TEMPDIR: Path, FASTA_ALLELES: dict[str, str], CONTROL_NAME: str, SAMPLE_NAME: str) -> None:
    for allele, sequence in FASTA_ALLELES.items():
        print(allele)
        midsv_sample = midsv.read_jsonl((Path(TEMPDIR, "midsv", f"{SAMPLE_NAME}_splice_{allele}.jsonl")))
        midsv_control = midsv.read_jsonl((Path(TEMPDIR, "midsv", f"{CONTROL_NAME}_splice_{allele}.jsonl")))
        coverage_sample = len(midsv_sample)
        coverage_control = len(midsv_control)
        cssplits_sample = [cs["CSSPLIT"].split(",") for cs in midsv_sample]
        cssplits_control = [cs["CSSPLIT"].split(",") for cs in midsv_control]
        left_idx, right_idx = set_indexes(sequence)
        count_5mer_sample = count_indels_5mer(cssplits_sample, left_idx, right_idx)
        count_5mer_control = count_indels_5mer(cssplits_control, left_idx, right_idx)
        sequence_errors = extract_sequence_errors(
            count_5mer_sample, count_5mer_control, coverage_sample, coverage_control
        )
        cssplits_sample_error_replaced = replace_errors_to_atmark(cssplits_sample, sequence_errors, left_idx, right_idx)
        cssplits_control_error_replaced = replace_errors_to_atmark(
            cssplits_control, sequence_errors, left_idx, right_idx
        )
        cssplits_sample_atmark_replaced = replace_atmark(cssplits_sample_error_replaced, sequence)
        cssplits_control_atmark_replaced = replace_atmark(cssplits_control_error_replaced, sequence)
        # Replace CSSPLIT
        cssplits_sample_corrected = [",".join(cs) for cs in cssplits_sample_atmark_replaced]
        cssplits_control_corrected = [",".join(cs) for cs in cssplits_control_atmark_replaced]
        for i, cssplits in enumerate(cssplits_sample_corrected):
            midsv_sample[i]["CSSPLIT"] = cssplits
        for i, cssplits in enumerate(cssplits_control_corrected):
            midsv_control[i]["CSSPLIT"] = cssplits
        # midsv.write_jsonl(midsv_control, Path(TEMPDIR, "midsv", f"{CONTROL_NAME}_splice_{allele}.jsonl"))
        # midsv.write_jsonl(midsv_sample, Path(TEMPDIR, "midsv", f"{SAMPLE_NAME}_splice_{allele}.jsonl"))



In [15]:
execute(TEMPDIR, FASTA_ALLELES, CONTROL_NAME, SAMPLE_NAME) 

deletion
control
inversion


IndexError: list index out of range

In [23]:
allele = "inversion"
sequence = FASTA_ALLELES[allele]

In [17]:
midsv_sample = midsv.read_jsonl((Path(TEMPDIR, "midsv", f"{SAMPLE_NAME}_splice_{allele}.jsonl")))
midsv_control = midsv.read_jsonl((Path(TEMPDIR, "midsv", f"{CONTROL_NAME}_splice_{allele}.jsonl")))
coverage_sample = len(midsv_sample)
coverage_control = len(midsv_control)
cssplits_sample = [cs["CSSPLIT"].split(",") for cs in midsv_sample]
cssplits_control = [cs["CSSPLIT"].split(",") for cs in midsv_control]
left_idx, right_idx = set_indexes(sequence)
count_5mer_sample = count_indels_5mer(cssplits_sample, left_idx, right_idx)
count_5mer_control = count_indels_5mer(cssplits_control, left_idx, right_idx)
sequence_errors = extract_sequence_errors(
    count_5mer_sample, count_5mer_control, coverage_sample, coverage_control
)
cssplits_sample_error_replaced = replace_errors_to_atmark(cssplits_sample, sequence_errors, left_idx, right_idx)
cssplits_control_error_replaced = replace_errors_to_atmark(
    cssplits_control, sequence_errors, left_idx, right_idx
)


In [24]:
cssplits_sample_atmark_replaced = replace_atmark(cssplits_sample_error_replaced, sequence)

IndexError: list index out of range

## replace_atmarkでエラーが発生

In [27]:
len(cssplits_sample_error_replaced[0]) == len(sequence)

True

In [28]:
cssplits = deepcopy(cssplits_sample_error_replaced)

In [48]:
random.seed(1)
cssplits_replaced = deepcopy(cssplits)
sequence_length = len(sequence)
for i in range(1, sequence_length - 1):
    cssplits_atmark = defaultdict(str)
    cssplits_sampling_key = defaultdict(list)
    cssplits_sampling_all = []
    flag_all_atmark = True
    for idx, cssplit in enumerate(cssplits):
        key = ",".join([cssplit[i - 1], cssplit[i + 1]])
        if cssplit[i] == "@":
            cssplits_atmark[idx] = key
        else:
            cssplits_sampling_key[key].append(cssplit[i])
            cssplits_sampling_all.append(cssplit[i])
            flag_all_atmark = False
    for idx, key in cssplits_atmark.items():
        if flag_all_atmark:
            cssplits_replaced[idx][i] = "N"
        elif cssplits_sampling_key[key]:
            cssplits_replaced[idx][i] = random.choice(cssplits_sampling_key[key])
        else:
            cssplits_replaced[idx][i] = random.choice(cssplits_sampling_all)

In [49]:
count = defaultdict(int)
for cs in cssplits:
    count[cs[i]] += 1
print(count)
print(len(cssplits_sampling_key))

defaultdict(<class 'int'>, {'=G': 1152, 'N': 299})
3


In [52]:
%%bash
# Stx2 deletion knockout
rm -rf DAJINResults/.tempdir/test-knockout/
time python misc/quickstart.py

processing test-knockout...


Traceback (most recent call last):
  File "/mnt/d/Research/DAJIN2/misc/quickstart.py", line 273, in <module>
    report.report_bam.output_bam_control(TEMPDIR, CONTROL_NAME, GENOME, GENOME_COODINATES, CHROME_SIZE, THREADS)
  File "/mnt/d/Research/DAJIN2/src/DAJIN2/core/report/report_bam.py", line 253, in output_bam_control
    sam = midsv.read_sam(intput_path_sam)
  File "/home/kuno/miniconda/lib/python3.10/site-packages/midsv/io.py", line 12, in read_sam
    sam = Path(path_of_sam).read_text().strip().split("\n")
  File "/home/kuno/miniconda/lib/python3.10/pathlib.py", line 1134, in read_text
    with self.open(mode='r', encoding=encoding, errors=errors) as f:
  File "/home/kuno/miniconda/lib/python3.10/pathlib.py", line 1119, in open
    return self._accessor.open(self, mode, buffering, encoding, errors,
FileNotFoundError: [Errno 2] No such file or directory: 'DAJINResults/.tempdir/test-knockout/sam/test_barcode30_map-ont_control.sam'

real	0m49.688s
user	1m14.422s
sys	0m45.999s
Trace

CalledProcessError: Command 'b'# Stx2 deletion knockout\nrm -rf DAJINResults/.tempdir/test-knockout/\ntime python misc/quickstart.py\n'' returned non-zero exit status 1.

## 今度はClusteringでエラーが発生

In [53]:
classif_sample = classification.classify_alleles(TEMPDIR, SAMPLE_NAME)
for classif in classif_sample:
    classif["SV"] = classification.detect_sv(classif["CSSPLIT"], threshold=50)

In [55]:
from src.DAJIN2.core.clustering import clustering
clust_sample = clustering.add_labels(classif_sample, TEMPDIR, CONTROL_NAME, FASTA_ALLELES, THREADS)

TypeError: iter expected at least 1 argument, got 0

In [59]:
from __future__ import annotations
from itertools import groupby
from copy import deepcopy
from collections import defaultdict
from pathlib import Path
import midsv

from src.DAJIN2.core.preprocess.correct_knockin import extract_knockin_loci
from src.DAJIN2.core.clustering.preprocess import replace_both_ends_n
from src.DAJIN2.core.clustering.make_score import make_score
from src.DAJIN2.core.clustering.annotate_score import annotate_score
from src.DAJIN2.core.clustering.reorder_labels import reorder_labels
from src.DAJIN2.core.clustering.return_labels import return_labels

def add_labels(classif_sample, TEMPDIR, CONTROL_NAME, FASTA_ALLELES: dict, THREADS: int = 1) -> list[dict[str]]:
    paths_midsv = list(Path(TEMPDIR, "midsv").glob(f"{CONTROL_NAME}_splice_*"))
    cssplits_control_by_alleles = defaultdict(list)
    for path_midsv in paths_midsv:
        midsv_control = midsv.read_jsonl(path_midsv)
        allele = path_midsv.stem.split("_")[-1]
        cssplits = [cs["CSSPLIT"].split(",") for cs in midsv_control]
        cssplits_control_by_alleles[allele] = cssplits
    knockin_alleles = extract_knockin_loci(TEMPDIR)
    labels_all = []
    max_label = 0
    classif_sample.sort(key=lambda x: (x["ALLELE"], x["SV"]))
    for (allele, _), group in groupby(classif_sample, key=lambda x: (x["ALLELE"], x["SV"])):
        print(allele)
        cssplits_control = cssplits_control_by_alleles[allele]
        cssplits_sample = [cs["CSSPLIT"].split(",") for cs in group]
        cssplits_control = replace_both_ends_n(cssplits_control)
        cssplits_sample = replace_both_ends_n(cssplits_sample)
        mutation_score = make_score(cssplits_control, cssplits_sample, knockin_alleles[allele])
        scores_control = annotate_score(cssplits_control, mutation_score)
        scores_sample = annotate_score(cssplits_sample, mutation_score)
        labels = return_labels(scores_sample, scores_control)
        labels_reorder = reorder_labels(labels, start=max_label)
        max_label = max(labels_reorder)
        labels_all.extend(labels_reorder)
    clust_sample = deepcopy(classif_sample)
    for clust, label in zip(clust_sample, labels_all):
        clust["LABEL"] = label
    return clust_sample

In [60]:
add_labels(classif_sample, TEMPDIR, CONTROL_NAME, FASTA_ALLELES, THREADS)

deletion
deletion
inversion


TypeError: iter expected at least 1 argument, got 0

In [61]:
paths_midsv = list(Path(TEMPDIR, "midsv").glob(f"{CONTROL_NAME}_splice_*"))
cssplits_control_by_alleles = defaultdict(list)
for path_midsv in paths_midsv:
    midsv_control = midsv.read_jsonl(path_midsv)
    allele = path_midsv.stem.split("_")[-1]
    cssplits = [cs["CSSPLIT"].split(",") for cs in midsv_control]
    cssplits_control_by_alleles[allele] = cssplits
knockin_alleles = extract_knockin_loci(TEMPDIR)
labels_all = []
max_label = 0
classif_sample.sort(key=lambda x: (x["ALLELE"], x["SV"]))

In [66]:
allele = "inversion"
sv = True

In [67]:
cssplits_control = cssplits_control_by_alleles[allele]
cssplits_sample = [cs["CSSPLIT"].split(",") for cs in classif_sample if cs["ALLELE"] == allele and cs["SV"] == sv]

In [68]:
cssplits_control = replace_both_ends_n(cssplits_control)

In [69]:
cssplits_sample = replace_both_ends_n(cssplits_sample)

TypeError: iter expected at least 1 argument, got 0

In [70]:
cssplits = deepcopy(cssplits_sample)

In [73]:
from __future__ import annotations
from collections import Counter
import numpy as np
from collections import defaultdict
from copy import deepcopy


def transpose(cssplits):
    return [list(cs) for cs in zip(*cssplits)]


def call_count(transpose_cssplits: list[list[str]]) -> list[dict[str:int]]:
    cssplit_counts = []
    for cssplit in transpose_cssplits:
        count = Counter(cssplit)
        count = dict(count)
        cssplit_counts.append(count)
    return cssplit_counts


def sampling(cnt: Counter, size: int):
    elements = []
    probs = []
    coverage = sum(cnt.values())
    for key, val in cnt.items():
        elements.append(key)
        probs.append(val / coverage)
    np.random.seed(1)
    samples = np.random.choice(a=elements, size=size, p=probs)
    return samples


In [109]:
transpose_cssplits = [list(cs) for cs in zip(*cssplits)]
d_samples = defaultdict(iter)
for i, cssplit in enumerate(transpose_cssplits):
    flag_no_N = all(True if cs != "N" else False for cs in cssplit)
    if flag_no_N:
            continue
    flag_all_N = all(True if cs == "N" else False for cs in cssplit)
    cnt = Counter(cssplit)
    size = cnt["N"]
    if not flag_all_N:
        del cnt["N"]
    samples = sampling(cnt, size)
    d_samples[i] = iter(samples)
cssplits_replaced = deepcopy(cssplits)
for i, cssplit in enumerate(cssplits_replaced):
    for j, cs in enumerate(cssplit):
        if cs != "N":
            break
        cssplits_replaced[i][j] = next(d_samples[j])
# for i, cssplit in enumerate(cssplits_replaced):
#     cssplit = cssplit[::-1]
#     for j, cs in enumerate(cssplit):
#         if cs != "N":
#             break
#         try:
#             cssplits_replaced[i][len(cssplit) - 1 - j] = next(d_samples[len(cssplit) - 1 - j])
#         except TypeError:
#             pass

In [89]:
cnt = Counter(transpose_cssplits[0])

In [110]:

def replace_both_ends_n(cssplits: list[list[str]]):
    transpose_cssplits = [list(cs) for cs in zip(*cssplits)]
    d_samples = defaultdict(iter)
    for i, cssplit in enumerate(transpose_cssplits):
        flag_no_N = all(True if cs != "N" else False for cs in cssplit)
        if flag_no_N:
                continue
        flag_all_N = all(True if cs == "N" else False for cs in cssplit)
        cnt = Counter(cssplit)
        size = cnt["N"]
        if not flag_all_N:
            del cnt["N"]
        samples = sampling(cnt, size)
        d_samples[i] = iter(samples)
    cssplits_replaced = deepcopy(cssplits)
    for i, cssplit in enumerate(cssplits_replaced):
        for j, cs in enumerate(cssplit):
            if cs != "N":
                break
            cssplits_replaced[i][j] = next(d_samples[j])
    for i, cssplit in enumerate(cssplits_replaced):
        cssplit = cssplit[::-1]
        for j, cs in enumerate(cssplit):
            if cs != "N":
                break
            try:
                cssplits_replaced[i][len(cssplit) - 1 - j] = next(d_samples[len(cssplit) - 1 - j])
            except TypeError:
                pass
    return cssplits_replaced

In [111]:
cssplits_sample = replace_both_ends_n(cssplits_sample)

## Retry

In [112]:
%%bash
# Stx2 deletion knockout
rm -rf DAJINResults/.tempdir/test-knockout/
time python misc/quickstart.py

processing test-knockout...



real	5m20.068s
user	5m33.203s
sys	0m50.345s


# 結果
- Stx2 knockoutはエラーはなくなった
- inversionのようにごく少数のアレルにおいて、全てが`N`である場合に補正がうまく行っていなかった
    - コードの可読性の低さを感じた
- 一方で、**クラスタが3つに別れず、1つになっていた**
    - 次回原因を探索する

# 次回の課題

+ `preprocess.correct_sequence_error.replace_atmark`のコードがわかりにくい
    + テストを用意してリファクタリングする
+ Stx2のクラスタリングがうまく行っていないので原因を探索する